# **Classification Problem**

In [ ]:
!pip install numpy pandas scikit-learn tensorflow==2.12.0 bayesian-optimization optuna pyswarm deap

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of bayesian-optimization to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import optuna
# ============================
# Load the Iris dataset
# ============================
iris = load_iris()  # Load the dataset
X, y = iris.data, iris.target  # Features and target variable

# ============================
# Split the dataset into training and testing sets
# ============================
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
random_state=42)

# ============================


# Function to create the Keras model
# ============================
def create_classification_model(optimizer='adam', activation='relu',
neurons=10):
    model = Sequential()  # Initialize a sequential model
    model.add(Dense(neurons, input_dim=X_train.shape[1],activation=activation))  # Hidden layer
    model.add(Dense(3, activation='softmax'))  # Output layer for 3 classes (Iris species)
    model.compile(loss='sparse_categorical_crossentropy',
    optimizer=optimizer, metrics=['accuracy'])  # Compile the model
    return model  # Return the created model
# ============================
# Wrap the model with KerasClassifier
# ============================
model = KerasClassifier(build_fn=create_classification_model, verbose=0)
# Wrap the Keras model for scikit-learn
# ============================
# Hyperparameter grid for optimization
# ============================
param_grid = {
    'optimizer': ['adam', 'rmsprop', 'sgd'],  # Different optimizers to test
    'activation': ['relu', 'tanh'],  # Activation functions to test
    'neurons': [10, 20, 30],  # Number of neurons in the hidden layer
    'batch_size': [5, 10],  # Batch sizes to test
    'epochs': [10, 20],  # Number of epochs for training
}

# ============================
# 1. Grid Search
# ============================
grid = GridSearchCV(estimator=model, param_grid=param_grid,
scoring='accuracy', cv=3)  # Initialize Grid Search
grid.fit(X_train, y_train)  # Fit the model on training data
print("Best parameters from Grid Search:", grid.best_params_)

<ipython-input-5-f8845e048c0a>:37: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_classification_model, verbose=0)


2/2 [==============================] - 0s 9ms/step


2/2 [==============================] - 0s 9ms/step


2/2 [==============================] - 0s 7ms/step
Best parameters from Grid Search: {'activation': 'relu', 'batch_size': 10, 'epochs': 20, 'neurons': 20, 'optimizer': 'sgd'}


In [ ]:
# ============================
# 2. Random Search
# ============================
random_search = RandomizedSearchCV(estimator=model,
param_distributions=param_grid, n_iter=10, scoring='accuracy', cv=3)  # Initialize Random Search
random_search.fit(X_train, y_train)  # Fit the model on training data
print("Best parameters from Random Search:", random_search.best_params_)

2/2 [==============================] - 0s 8ms/step
Best parameters from Random Search: {'optimizer': 'sgd', 'neurons': 20, 'epochs': 20, 'batch_size': 5, 'activation': 'tanh'}


In [ ]:
# ============================
# 3. Hyperband (using Optuna)
# ============================
def objective(trial):
    # Define hyperparameters to optimize
    optimizer = trial.suggest_categorical('optimizer', ['adam', 'rmsprop', 'sgd'])  # Optimizer
    activation = trial.suggest_categorical('activation', ['relu', 'tanh'])  # Activation function
    neurons = trial.suggest_int('neurons', 10, 30)  # Number of neurons
    batch_size = trial.suggest_categorical('batch_size', [5, 10])  # Batch size
    # Create the model with the suggested hyperparameters
    model = create_classification_model(optimizer=optimizer, activation=activation,neurons=neurons)

    # Train the model
    model.fit(X_train, y_train, batch_size=batch_size, epochs=10, verbose=0)
    # Evaluate the model on the test set
    accuracy = model.evaluate(X_test, y_test, verbose=0)[1]  # Get the accuracy
    return accuracy  # Return the accuracy for optimization

# Create and optimize the study
study = optuna.create_study(direction='maximize')  # Create a study to maximize accuracy
study.optimize(objective, n_trials=10)  # Optimize the objective function over 10 trials
print("Best parameters from Hyperband (Optuna):", study.best_params)


[I 2024-09-24 07:22:04,999] A new study created in memory with name: no-name-6a2005ca-a8b7-4a8d-a32b-5230017ed019
[I 2024-09-24 07:22:06,723] Trial 0 finished with value: 0.7333333492279053 and parameters: {'optimizer': 'sgd', 'activation': 'relu', 'neurons': 26, 'batch_size': 5}. Best is trial 0 with value: 0.7333333492279053.
[I 2024-09-24 07:22:08,446] Trial 1 finished with value: 0.800000011920929 and parameters: {'optimizer': 'adam', 'activation': 'tanh', 'neurons': 20, 'batch_size': 5}. Best is trial 1 with value: 0.800000011920929.
[I 2024-09-24 07:22:09,778] Trial 2 finished with value: 0.9666666388511658 and parameters: {'optimizer': 'sgd', 'activation': 'relu', 'neurons': 23, 'batch_size': 5}. Best is trial 2 with value: 0.9666666388511658.
[I 2024-09-24 07:22:12,078] Trial 3 finished with value: 0.9666666388511658 and parameters: {'optimizer': 'rmsprop', 'activation': 'relu', 'neurons': 10, 'batch_size': 5}. Best is trial 2 with value: 0.9666666388511658.
[I 2024-09-24 07:22

Best parameters from Hyperband (Optuna): {'optimizer': 'sgd', 'activation': 'tanh', 'neurons': 20, 'batch_size': 10}


In [ ]:
from bayes_opt import BayesianOptimization

# ============================
# Function to optimize with Bayesian Optimization
# ============================
def bayesian_opt(neurons, optimizer_idx, activation_idx, batch_size):
    # Define options for optimizers and activation functions
    optimizer_options = ['adam', 'rmsprop', 'sgd']  # List of optimizer options
    activation_options = ['relu', 'tanh']  # List of activation function options

    # Map indices to actual optimizer and activation function
    optimizer = optimizer_options[int(optimizer_idx)]  # Select optimizer based on index
    activation = activation_options[int(activation_idx)]  # Select activation function based on index
    # Create and compile the Keras model
    model = create_classification_model(optimizer=optimizer, activation=activation, neurons=int(neurons))

    # Train the model on the training set
    model.fit(X_train, y_train, batch_size=int(batch_size), epochs=10, verbose=0)

    # Evaluate the model on the test set and return the accuracy
    accuracy = model.evaluate(X_test, y_test, verbose=0)[1]
    return accuracy  # Return the achieved accuracy

# ============================
# Bounds for Bayesian Optimization
# ============================
pbounds = {
    'neurons': (10, 30),  # Number of neurons in the hidden layer
    'optimizer_idx': (0, 2),  # Indices for optimizers (0: 'adam', 1: 'rmsprop', 2: 'sgd')
    'activation_idx': (0, 1),  # Indices for activation functions (0: 'relu', 1: 'tanh')
    'batch_size': (5, 10),  # Batch size to test
}

# ============================
# Initialize and run Bayesian Optimization
# ============================
optimizer = BayesianOptimization(f=bayesian_opt, pbounds=pbounds, random_state=1)  # Create the optimizer
optimizer.maximize(init_points=5, n_iter=10)  # Run optimization with initial points and iterations
print("Best parameters from BO-GP:", optimizer.max)


|   iter    |  target   | activa... | batch_... |  neurons  | optimi... |
-------------------------------------------------------------------------
| 1         | 0.4       | 0.417     | 8.602     | 10.0      | 0.6047    |
| 2         | 0.8333    | 0.1468    | 5.462     | 13.73     | 0.6911    |
| 3         | 0.8333    | 0.3968    | 7.694     | 18.38     | 1.37      |
| 4         | 0.3667    | 0.2045    | 9.391     | 10.55     | 1.341     |
| 5         | 0.7       | 0.4173    | 7.793     | 12.81     | 0.3962    |
| 6         | 0.9667    | 1.0       | 5.0       | 22.99     | 0.0       |
| 7         | 0.9       | 0.0       | 7.665     | 28.18     | 0.0       |
| 8         | 0.7333    | 1.0       | 10.0      | 24.36     | 2.0       |
| 9         | 1.0       | 1.0       | 5.0       | 30.0      | 2.0       |
| 10        | 0.9       | 0.0       | 5.0       | 26.23     | 2.0       |
| 11        | 0.9       | 1.0       | 5.0       | 30.0      | 0.0       |
| 12        | 0.8       | 1.0       | 

In [ ]:
!pip install pyswarm

In [ ]:
from pyswarm import pso
# ============================
# Particle Swarm Optimization (PSO)
# ============================
from pyswarm import pso  # Import the PSO function

def pso_objective(params):
    # Extract parameters from the input array
    neurons, batch_size = int(params[0]), int(params[1])
    optimizer = 'adam'  # Fixed optimizer for simplification
    activation = 'relu'  # Fixed activation for simplification
    # Create and compile the Keras model
    model = create_classification_model(optimizer=optimizer, activation=activation, neurons=neurons)

    # Train the model
    model.fit(X_train, y_train, batch_size=batch_size, epochs=10, verbose=0)

    # Evaluate the model on the test set and return the negative accuracy (to minimize)
    accuracy = model.evaluate(X_test, y_test, verbose=0)[1]
    return -accuracy  # Return negative accuracy for minimization

# Define bounds for the PSO parameters
lb = [10, 5]  # Lower bounds for neurons and batch size
ub = [30, 10]  # Upper bounds for neurons and batch size

# Run Particle Swarm Optimization
best_params, _ = pso(pso_objective, lb, ub, swarmsize=10, maxiter=5)  # Optimize with PSO
print("Best parameters from PSO:", best_params)


Stopping search: maximum iterations reached --> 5
Best parameters from PSO: [13.51565461  7.88549923]


# **Forecasting California Housing Prices with LSTM, GRU, RNN,and CNN Models (Sequential Problem)**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, Dropout, SimpleRNN, Conv1D, Flatten
from keras.callbacks import EarlyStopping
import optuna
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error, r2_score

# ============================
# Load and Prepare the Dataset
# ============================
# Load California housing dataset
data = fetch_california_housing()
X = data.data  # Features
y = data.target  # Target variable

# Create sequences for time series modeling
def create_sequences(X, y, time_steps=10):
    X_seq, y_seq = [], []
    for i in range(len(X) - time_steps):
        X_seq.append(X[i:i + time_steps])  # Append sequences of features
        y_seq.append(y[i + time_steps])  # Append the target for the next time step
    return np.array(X_seq), np.array(y_seq)

# Create sequences
X_seq, y_seq = create_sequences(X, y)

# Reshape X for LSTM/GRU/RNN: (samples, time steps, features)
X_seq = X_seq.reshape((X_seq.shape[0], X_seq.shape[1], X_seq.shape[2]))

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

# Scale the target variable for better training performance
scaler = StandardScaler()
y_train = scaler.fit_transform(y_train.reshape(-1, 1)).flatten()
y_test = scaler.transform(y_test.reshape(-1, 1)).flatten()

# Early stopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

# ============================
# Model Creation Functions
# ============================
# Function to create an LSTM model
def create_lstm_model(optimizer='adam', lstm_units=50, dropout_rate=0.2):
    model = Sequential()
    model.add(LSTM(lstm_units, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(dropout_rate))  # Dropout for regularization
    model.add(Dense(1))  # Output layer for regression
    model.compile(loss='mean_squared_error', optimizer=optimizer)  # Compile model
    return model

# Function to create a GRU model
def create_gru_model(optimizer='adam', gru_units=50, dropout_rate=0.2):
    model = Sequential()
    model.add(GRU(gru_units, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(dropout_rate))  # Dropout for regularization
    model.add(Dense(1))  # Output layer for regression
    model.compile(loss='mean_squared_error', optimizer=optimizer)  # Compile model
    return model

# Function to create a Simple RNN model
def create_rnn_model(optimizer='adam', rnn_units=50, dropout_rate=0.2):
    model = Sequential()
    model.add(SimpleRNN(rnn_units, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(dropout_rate))  # Dropout for regularization
    model.add(Dense(1))  # Output layer for regression
    model.compile(loss='mean_squared_error', optimizer=optimizer)  # Compile model
    return model

# Function to create a 1D CNN model
def create_cnn_model(optimizer='adam', filters=32, kernel_size=3, dropout_rate=0.2):
    model = Sequential()
    model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(dropout_rate))  # Dropout for regularization
    model.add(Flatten())  # Flatten for the Dense layer
    model.add(Dense(1))  # Output layer for regression
    model.compile(loss='mean_squared_error', optimizer=optimizer)  # Compile model
    return model

# ============================
# Model Evaluation Function
# ============================
def evaluate_model(model):
    predictions = model.predict(X_test)  # Generate predictions on test set
    predictions = scaler.inverse_transform(predictions)  # Inverse scaling to original scale
    mse = mean_squared_error(y_test, predictions)  # Calculate Mean Squared Error
    r2 = r2_score(y_test, predictions)  # Calculate R² score
    accuracy_percentage = r2 * 100  # Convert R² to percentage
    return mse, accuracy_percentage  # Return both metrics

# ============================
# Hyperparameter Tuning Functions
# ============================
# Hyperparameter tuning for LSTM
def objective_lstm(trial):
    optimizer = trial.suggest_categorical('optimizer', ['adam', 'sgd'])  # Choose optimizer
    lstm_units = trial.suggest_int('lstm_units', 10, 100)  # Number of LSTM units
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)  # Dropout rate

    model = create_lstm_model(optimizer=optimizer, lstm_units=lstm_units, dropout_rate=dropout_rate)
    model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1, callbacks=[early_stopping])  # Fit model

    mse, accuracy = evaluate_model(model)  # Evaluate model
    return mse  # Minimize MSE

# Hyperparameter tuning for GRU
def objective_gru(trial):
    optimizer = trial.suggest_categorical('optimizer', ['adam', 'sgd'])  # Choose optimizer
    gru_units = trial.suggest_int('gru_units', 10, 100)  # Number of GRU units
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)  # Dropout rate

    model = create_gru_model(optimizer=optimizer, gru_units=gru_units, dropout_rate=dropout_rate)
    model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1, callbacks=[early_stopping])  # Fit model

    mse, accuracy = evaluate_model(model)  # Evaluate model
    return mse  # Minimize MSE

# Hyperparameter tuning for RNN
def objective_rnn(trial):
    optimizer = trial.suggest_categorical('optimizer', ['adam', 'sgd'])  # Choose optimizer
    rnn_units = trial.suggest_int('rnn_units', 10, 100)  # Number of RNN units
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)  # Dropout rate

    model = create_rnn_model(optimizer=optimizer, rnn_units=rnn_units, dropout_rate=dropout_rate)
    model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1, callbacks=[early_stopping])  # Fit model

    mse, accuracy = evaluate_model(model)  # Evaluate model
    return mse  # Minimize MSE

# Hyperparameter tuning for CNN
def objective_cnn(trial):
    optimizer = trial.suggest_categorical('optimizer', ['adam', 'sgd'])  # Choose optimizer
    filters = trial.suggest_int('filters', 16, 64)  # Number of filters
    kernel_size = trial.suggest_int('kernel_size', 2, 5)  # Kernel size
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)  # Dropout rate

    model = create_cnn_model(optimizer=optimizer, filters=filters, kernel_size=kernel_size, dropout_rate=dropout_rate)
    model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1, callbacks=[early_stopping])  # Fit model

    mse, accuracy = evaluate_model(model)  # Evaluate model
    return mse  # Minimize MSE

# ============================
# Optuna Studies for Hyperparameter Optimization
# ============================
# Optuna study for LSTM
study_lstm = optuna.create_study(direction='minimize')
study_lstm.optimize(objective_lstm, n_trials=5)  # Optimize LSTM parameters
best_lstm_mse, best_lstm_accuracy = evaluate_model(create_lstm_model(**study_lstm.best_params))
print("Best parameters for LSTM:", study_lstm.best_params)
print(f"LSTM - MSE: {best_lstm_mse:.2f}, Accuracy Percentage: {best_lstm_accuracy:.2f}%")

# Optuna study for GRU
study_gru = optuna.create_study(direction='minimize')
study_gru.optimize(objective_gru, n_trials=5)  # Optimize GRU parameters
best_gru_mse, best_gru_accuracy = evaluate_model(create_gru_model(**study_gru.best_params))
print("Best parameters for GRU:", study_gru.best_params)
print(f"GRU - MSE: {best_gru_mse:.2f}, Accuracy Percentage: {best_gru_accuracy:.2f}%")

# Optuna study for


[I 2024-09-30 16:36:23,577] A new study created in memory with name: no-name-e88a1b2f-d869-4153-97da-b1f2302bb8b3


Epoch 1/20
516/516 [==============================] - 12s 10ms/step - loss: 1.0129
Epoch 2/20
516/516 [==============================] - 4s 8ms/step - loss: 0.9959
Epoch 3/20
516/516 [==============================] - 4s 8ms/step - loss: 0.9952
Epoch 4/20
516/516 [==============================] - 4s 7ms/step - loss: 0.9960
Epoch 5/20
516/516 [==============================] - 4s 8ms/step - loss: 0.9965
Epoch 6/20
129/129 [==============================] - 1s 3ms/step


[I 2024-09-30 16:36:59,546] Trial 0 finished with value: 5.110640497218157 and parameters: {'optimizer': 'sgd', 'lstm_units': 16, 'dropout_rate': 0.18557598946423323}. Best is trial 0 with value: 5.110640497218157.


Epoch 1/20
516/516 [==============================] - 7s 8ms/step - loss: 1.0682
Epoch 2/20
516/516 [==============================] - 5s 10ms/step - loss: 1.0241
Epoch 3/20
516/516 [==============================] - 4s 8ms/step - loss: 1.0072
Epoch 4/20
516/516 [==============================] - 4s 8ms/step - loss: 1.0021
Epoch 5/20
516/516 [==============================] - 5s 10ms/step - loss: 0.9935
Epoch 6/20
516/516 [==============================] - 7s 14ms/step - loss: 0.9439
Epoch 7/20
516/516 [==============================] - 7s 13ms/step - loss: 0.8148
Epoch 8/20
516/516 [==============================] - 7s 13ms/step - loss: 0.7795
Epoch 9/20
516/516 [==============================] - 5s 11ms/step - loss: 0.7312
Epoch 10/20
516/516 [==============================] - 8s 16ms/step - loss: 0.7323
Epoch 11/20
516/516 [==============================] - 7s 13ms/step - loss: 0.7164
Epoch 12/20
516/516 [==============================] - 7s 14ms/step - loss: 0.7235
Epoch 13/20
516/

[I 2024-09-30 16:38:55,172] Trial 1 finished with value: 4.278432363521988 and parameters: {'optimizer': 'adam', 'lstm_units': 18, 'dropout_rate': 0.10831512149766637}. Best is trial 1 with value: 4.278432363521988.


...

Epoch 1/20
516/516 [==============================] - 7s 9ms/step - loss: 1.0912
Epoch 2/20
516/516 [==============================] - 5s 11ms/step - loss: 1.0325
Epoch 3/20
516/516 [==============================] - 5s 9ms/step - loss: 1.0080
Epoch 4/20
516/516 [==============================] - 4s 9ms/step - loss: 1.0011
Epoch 5/20
516/516 [==============================] - 5s 10ms/step - loss: 0.9933
Epoch 6/20
516/516 [==============================] - 4s 8ms/step - loss: 0.9911
Epoch 7/20
516/516 [==============================] - 4s 8ms/step - loss: 1.0029
Epoch 8/20
516/516 [==============================] - 5s 11ms/step - loss: 0.9783
Epoch 9/20
516/516 [==============================] - 4s 9ms/step - loss: 0.9066
Epoch 10/20
516/516 [==============================] - 4s 9ms/step - loss: 0.8368
Epoch 11/20
516/516 [==============================] - 5s 10ms/step - loss: 0.7849
Epoch 12/20
516/516 [==============================] - 4s 9ms/step - loss: 0.7749
Epoch 13/20
516/516 [

[I 2024-09-30 16:49:16,376] Trial 3 finished with value: 5.97705668216411 and parameters: {'optimizer': 'adam', 'gru_units': 13, 'dropout_rate': 0.11033658792476855}. Best is trial 2 with value: 4.560185350210599.


Epoch 1/20
516/516 [==============================] - 9s 12ms/step - loss: 1.2158
Epoch 2/20
516/516 [==============================] - 6s 13ms/step - loss: 1.0072
Epoch 3/20
516/516 [==============================] - 6s 11ms/step - loss: 0.8340
Epoch 4/20
516/516 [==============================] - 7s 13ms/step - loss: 0.7456
Epoch 5/20
516/516 [==============================] - 6s 11ms/step - loss: 0.6747
Epoch 6/20
516/516 [==============================] - 7s 14ms/step - loss: 0.6276
Epoch 7/20
516/516 [==============================] - 6s 12ms/step - loss: 0.5917
Epoch 8/20
516/516 [==============================] - 7s 13ms/step - loss: 0.5796
Epoch 9/20
516/516 [==============================] - 6s 11ms/step - loss: 0.5806
Epoch 10/20
516/516 [==============================] - 7s 14ms/step - loss: 0.5594
Epoch 11/20
516/516 [==============================] - 6s 11ms/step - loss: 0.5915
Epoch 12/20
516/516 [==============================] - 6s 12ms/step - loss: 0.5704
Epoch 13/20
5

[I 2024-09-30 16:51:44,281] Trial 4 finished with value: 4.429535527641304 and parameters: {'optimizer': 'adam', 'gru_units': 85, 'dropout_rate': 0.1689673689977081}. Best is trial 4 with value: 4.429535527641304.


129/129 [==============================] - 1s 4ms/step
Best parameters for GRU: {'optimizer': 'adam', 'gru_units': 85, 'dropout_rate': 0.1689673689977081}
GRU - MSE: 21.28, Accuracy Percentage: -1978.56%


# **Cross Validation (K-Fold, Stratified K-fold, Repeated K-fold)**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold, StratifiedKFold, RepeatedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense

# ============================
# Load the Iris dataset
# ============================
data = load_iris()  # Load the Iris dataset from sklearn
X = data.data  # Features (sepal length, sepal width, etc.)
y = data.target  # Target variable (species of iris)

# ============================
# Scale features
# ============================
scaler = StandardScaler()  # Initialize the scaler
X_scaled = scaler.fit_transform(X)  # Scale features to have mean=0 and variance=1

# ============================
# Function to create the ANN model
# ============================
def create_ann_model():
    model = Sequential()  # Initialize a sequential model
    model.add(Dense(10, activation='relu', input_shape=(X_scaled.shape[1],)))  # Hidden layer with 10 neurons
    model.add(Dense(3, activation='softmax'))  # Output layer with 3 neurons (for 3 classes) using softmax activation
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])  # Compile the model
    return model  # Return the created model

# ============================
# K-Fold Cross-Validation
# ============================
def k_fold_cross_validation(X, y, n_splits=5):
    kf = KFold(n_splits=n_splits)  # Initialize K-Fold cross-validation
    accuracies = []  # List to store accuracies for each fold

    # Iterate through each fold
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]  # Split the data into training and testing sets
        y_train, y_test = y[train_index], y[test_index]  # Split the labels accordingly

        model = create_ann_model()  # Create a new ANN model
        model.fit(X_train, y_train, epochs=50, batch_size=5, verbose=0)  # Train the model
        y_pred = np.argmax(model.predict(X_test), axis=1)  # Make predictions on the test set
        accuracies.append(accuracy_score(y_test, y_pred))  # Calculate accuracy and store it

    print("K-Fold Cross-Validation Accuracy: ", np.mean(accuracies))  # Print the average accuracy

# ============================
# Stratified K-Fold Cross-Validation
# ============================
def stratified_k_fold_cross_validation(X, y, n_splits=5):
    skf = StratifiedKFold(n_splits=n_splits)  # Initialize Stratified K-Fold cross-validation
    accuracies = []  # List to store accuracies for each fold

    # Iterate through each fold
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]  # Split the data into training and testing sets
        y_train, y_test = y[train_index], y[test_index]  # Split the labels accordingly

        model = create_ann_model()  # Create a new ANN model
        model.fit(X_train, y_train, epochs=50, batch_size=5, verbose=0)  # Train the model
        y_pred = np.argmax(model.predict(X_test), axis=1)  # Make predictions on the test set
        accuracies.append(accuracy_score(y_test, y_pred))  # Calculate accuracy and store it

    print("Stratified K-Fold Cross-Validation Accuracy: ", np.mean(accuracies))  # Print the average accuracy

# ============================
# Repeated K-Fold Cross-Validation
# ============================
def repeated_k_fold_cross_validation(X, y, n_splits=5, n_repeats=3):
    rkf = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats)  # Initialize Repeated K-Fold cross-validation
    accuracies = []  # List to store accuracies for each fold

    # Iterate through each fold
    for train_index, test_index in rkf.split(X):
        X_train, X_test = X[train_index], X[test_index]  # Split the data into training and testing sets
        y_train, y_test = y[train_index], y[test_index]  # Split the labels accordingly

        model = create_ann_model()  # Create a new ANN model
        model.fit(X_train, y_train, epochs=50, batch_size=5, verbose=0)  # Train the model
        y_pred = np.argmax(model.predict(X_test), axis=1)  # Make predictions on the test set
        accuracies.append(accuracy_score(y_test, y_pred))  # Calculate accuracy and store it

    print("Repeated K-Fold Cross-Validation Accuracy: ", np.mean(accuracies))  # Print the average accuracy

# ============================
# Run cross-validation methods
# ============================
k_fold_cross_validation(X_scaled, y)  # Execute K-Fold cross-validation
stratified_k_fold_cross_validation(X_scaled, y)  # Execute Stratified K-Fold cross-validation
repeated_k_fold_cross_validation(X_scaled, y)  # Execute Repeated K-Fold cross-validation


1/1 [==============================] - 0s 56ms/step


1/1 [==============================] - 0s 145ms/step


1/1 [==============================] - 0s 136ms/step
K-Fold Cross-Validation Accuracy:  0.8200000000000001
1/1 [==============================] - 0s 86ms/step
Stratified K-Fold Cross-Validation Accuracy:  0.9199999999999999
1/1 [==============================] - 0s 59ms/step
Repeated K-Fold Cross-Validation Accuracy:  0.9311111111111112


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.datasets import fetch_california_housing

# Load the California Housing dataset
data = fetch_california_housing()
X = data.data  # Features
y = data.target  # Target variable (housing prices)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the data (important for ANN)
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

y_train = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()
y_test = scaler_y.transform(y_test.reshape(-1, 1)).flatten()


import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout

# Define the ANN model
def create_ann_model():
    model = Sequential()

    # Input layer with 64 neurons and ReLU activation
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))

    # Hidden layer with 32 neurons and ReLU activation
    model.add(Dense(32, activation='relu'))

    # Output layer with 1 neuron (regression output)
    model.add(Dense(1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    return model

# Initialize and view the model summary
model = create_ann_model()
model.summary()



from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV

# Wrap the model in a KerasRegressor for use in GridSearchCV
model = KerasRegressor(build_fn=create_ann_model, verbose=0)

# Define the grid of hyperparameters to search
param_grid = {
    'batch_size': [16, 32, 64],
    'epochs': [50, 100]
}

# Perform Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3)
grid_result = grid.fit(X_train, y_train)

# Print best parameters and results
print(f"Best Parameters: {grid_result.best_params_}")
print(f"Best MSE: {-grid_result.best_score_:.4f}")



import optuna
from keras.layers import Dropout

# Objective function for Optuna
def objective(trial):
    # Define hyperparameters to tune
    num_neurons = trial.suggest_int('num_neurons', 16, 128)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2)

    # Create the model with suggested hyperparameters
    model = Sequential()
    model.add(Dense(num_neurons, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_neurons//2, activation='relu'))
    model.add(Dense(1))

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    # Train the model
    model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

    # Evaluate the model
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    return mse

# Run the optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

# Best parameters from Optuna
print(f"Best hyperparameters: {study.best_params}")
print(f"Best MSE: {study.best_value:.4f}")



# Train the model using the best hyperparameters from GridSearch or Optuna
best_model = create_ann_model()
best_model.fit(X_train, y_train, epochs=grid_result.best_params_['epochs'], batch_size=grid_result.best_params_['batch_size'], verbose=1)

# Predict on the test set
y_pred = best_model.predict(X_test)

# Inverse transform the scaled target variable
y_pred = scaler_y.inverse_transform(y_pred)
y_test_orig = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Evaluate the model
mse = mean_squared_error(y_test_orig, y_pred)
r2 = r2_score(y_test_orig, y_pred)

print(f"Mean Squared Error: {mse:.4f}")
print(f"R² Score: {r2:.4f}")


Model: "sequential_37"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_62 (Dense)            (None, 64)                576       
                                                                 
 dense_63 (Dense)            (None, 32)                2080      
                                                                 
 dense_64 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2,689
Trainable params: 2,689
Non-trainable params: 0
_________________________________________________________________


<ipython-input-3-379aace86f22>:59: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasRegressor(build_fn=create_ann_model, verbose=0)
[I 2024-09-30 17:27:53,584] A new study created in memory with name: no-name-c2a5e5c5-5ddc-46f3-a4a2-0c1b4bef1367


Best Parameters: {'batch_size': 16, 'epochs': 100}
Best MSE: 0.2120
129/129 [==============================] - 0s 1ms/step


[I 2024-09-30 17:28:43,165] Trial 0 finished with value: 0.21475482409303853 and parameters: {'num_neurons': 98, 'dropout_rate': 0.32701832963603594, 'learning_rate': 0.0049096427666714074}. Best is trial 0 with value: 0.21475482409303853.


129/129 [==============================] - 1s 2ms/step


...

129/129 [==============================] - 0s 2ms/step


[I 2024-09-30 17:38:13,829] Trial 8 finished with value: 0.23294319970662566 and parameters: {'num_neurons': 92, 'dropout_rate': 0.48620873757627836, 'learning_rate': 0.008202627498596298}. Best is trial 4 with value: 0.19405087059895726.


129/129 [==============================] - 0s 2ms/step


[I 2024-09-30 17:39:37,030] Trial 9 finished with value: 0.20931085855701523 and parameters: {'num_neurons': 52, 'dropout_rate': 0.16347466088343315, 'learning_rate': 0.003246329370839119}. Best is trial 4 with value: 0.19405087059895726.


Best hyperparameters: {'num_neurons': 114, 'dropout_rate': 0.2513072437706121, 'learning_rate': 0.0025027069220994167}
Best MSE: 0.1941
Epoch 1/100
1032/1032 [==============================] - 4s 3ms/step - loss: 0.3862
Epoch 2/100
1032/1032 [==============================] - 2s 2ms/step - loss: 0.2972
Epoch 3/100
1032/1032 [==============================] - 2s 2ms/step - loss: 0.2577
Epoch 4/100
1032/1032 [==============================] - 2s 2ms/step - loss: 0.2668
Epoch 5/100
1032/1032 [==============================] - 2s 2ms/step - loss: 0.2327
Epoch 6/100
1032/1032 [==============================] - 4s 3ms/step - loss: 0.2317
Epoch 7/100
1032/1032 [==============================] - 3s 3ms/step - loss: 0.2491
Epoch 8/100
1032/1032 [==============================] - 2s 2ms/step - loss: 0.2248
Epoch 9/100
1032/1032 [==============================] - 3s 3ms/step - loss: 0.2150
Epoch 10/100
1032/1032 [==============================] - 3s 3ms/step - loss: 0.2117
Epoch 11/100
1032/1032 